In [10]:
import requests
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
import os
import re
from urllib.request import urlopen
import numpy as np
from urllib.request import Request, urlopen
import nltk
from nltk.corpus import stopwords
import unicodedata
nltk.download('stopwords')
from scrapy.http import TextResponse
import datetime
from scrapy import Selector

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\robert.lowe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
df = pd.read_csv('977_missing.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 8 columns):
Unnamed: 0               977 non-null int64
Cook_time                977 non-null object
Ingredients              977 non-null object
Link                     977 non-null object
Number of ingredients    977 non-null object
Serving                  977 non-null object
Title                    630 non-null object
Image_link               0 non-null float64
dtypes: float64(1), int64(1), object(6)
memory usage: 61.1+ KB


In [9]:
df['Ingredients'] = df['Ingredients'].replace('none', np.nan)

In [11]:
def jamie(url):
    try:
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, "html.parser")

        # Title 
        recipe = soup.find("h1", class_="hidden-xs")
        recipe = recipe.text
        
        # Feeds
        Feeds = soup.find("div", class_="recipe-detail serves")
        Feeds = Feeds.text
        Feeds = Feeds.replace('\nCooks In', '')
        Feeds = Feeds.replace('                                                    ', '')
        Feeds = Feeds.replace('\n', '')
        Feeds = Feeds.replace('    ','')
        
        # Time
        Time = soup.find("div", class_="recipe-detail time")
        Time = Time.text
        Time = Time.replace('\nCooks In', '')
        Time = Time.replace('                                                    ', '')
        
        # Ingredients 
        recipes = []
        for p in soup.findAll("ul", class_="ingred-list"):
            if p.find("li", class_="ingred-heading"):
                pass
            recipes.append(p.get_text())
        y = [re.sub(",", "", x) for x in recipes]
        y = ",".join(y)
        y = y.replace('    ', ' ')
        y = y.replace('    ', ' ')
        y = y.replace('    ', ' ')
        y = y.replace('    ', ' ')
        y = y.replace('    ', ' ')
        y = y.replace('   ', ' ')
        y = y.replace('\n\n',', ')
        y = y.replace(' TO SERVE, ',' ')
        y = y.replace('  \n, ','')
        y = y.replace('  \n', '')
        y = y.replace('  ,  ', ', ')
        y = y.replace('   ',' ')
        #y= y.replace('  ', ' ')

        y = y[3:]
        mylist = y.split(',')
        
        #ignore this link
        title = soup.findAll("li", class_="ingred-heading")
        title = [x.text for x in title]
        clean = []
        for z in title:   
            b = z.replace('\n                    ', '').replace('                  ','')
            clean.append(b)
        title1 = [x.replace('                ', '') for x in clean]
        ingredients = [x for x in mylist if x not in title1]
        ingredients = (',').join(ingredients)

        # Number of Ingredients
        ingred_count = ingredients.count(',')
        ingred_count = ingred_count + 1
        
        # image 
        image = soup.find("img", class_="")

        image = str(image)
        image = image.split('src="')
        image = image[1].replace(',"/>','')

        # Create a dataframe
        d = {
            'Title': recipe, 
             "Serving": Feeds, 
             'Cook_time': Time, 
             'Ingredients': ingredients,
            "Number of ingredients" : ingred_count,
            "Image_link": image,
             "Link": url

            }
        df = pd.DataFrame(data=d, index=[0])

        return df
    except:
           
        pass

        # Create a dataframe
        d = {
            'Title': np.nan, 
             "Serving": 'none', 
             'Cook_time': 'none', 
             'Ingredients': np.nan,
            "Number of ingredients" : 'none',
            "Image_link": np.nan,
             "Link": url

            }
        df = pd.DataFrame(data=d, index=[0])

        return df


In [12]:
df.head()

,Unnamed: 0,Cook_time,Ingredients,Link,Number of ingredients,Serving,Title,Image_link
0,582,none,NaN,https://www.jamieoliver.com/recipes/beetroot-r...,none,none,none,NaN
1,584,none,NaN,https://www.jamieoliver.com/recipes/vegetable-...,none,none,none,NaN
2,586,none,NaN,https://www.jamieoliver.com/recipes/vegetable-...,none,none,none,NaN
3,588,none,NaN,https://www.jamieoliver.com/recipes/beetroot-r...,none,none,none,NaN
4,590,none,NaN,https://www.jamieoliver.com/recipes/vegetable-...,none,none,none,NaN


In [13]:
df['Ingredients'].isnull().sum()

977

In [14]:
import datetime

In [ ]:
while df['Ingredients'].isnull().sum() >0:
    df= df.Link
    x1 = [jamie(x) for x in df]
    df = pd.concat(x1)
    ing  = df[df['Ingredients'].notnull()]
    ing.to_csv('jamie_recipies_{}.csv'.format(datetime.datetime.now().timestamp()))
    df  = df[df['Ingredients'].isnull()]
    print(df['Ingredients'].isnull().sum())



918
860
836
795
722
681
607
564
523
505
444
435
397
376
371
371
350
350
329
308
287
287
267
246
246
246
230
230
210
210
202
202
202
202
202
185
185
185
171
171
171
171
163
163
163
144
144
142
142
142
142
142
133
133
112
112
112
112
106
98
98
98
96
96
96
96
96
81
81
81
81
81
81
81
74
74
74
74
74
74
74
74
74
74
74
74
74
74
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71


In [9]:
import requests
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
import os
import requests
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup as bs
import re
from urllib.request import urlopen
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
from urllib.request import Request, urlopen
import nltk
from nltk.corpus import stopwords
import unicodedata
from bs4 import BeautifulSoup
nltk.download('stopwords')
from scrapy.http import TextResponse
import requests
import datetime
from scrapy import Selector

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\robert.lowe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
